In [14]:
# %load lstm_prob.py
"""
Created on Mon Nov 13 19:53:19 2017

@author: merry jane

1、训练lstm 生成文本模型

"""

from keras.models import Sequential
from keras.layers import Dense, Activation,LSTM
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import keras
import numpy as np
#from keras.datasets import imdb
from keras import backend as K
import re
import jieba
import pickle
import data_helper




def build_lstm_model(batch_size=256):
    '''
    arg:
        char_indices:输入文本的词语索引字典
        maxlen:每一行文本按词语最大长度
        
    
    模型如图所示，利用keras搭建，tensorflow作为backend
    ________________________________________________________________
    Layer (type)                 Output Shape              Param #   
    =================================================================
    lstm_24 (LSTM)               (None, 256)               13986816  
    _________________________________________________________________
    dense_14 (Dense)             (None, 13402)             3444314   
    _________________________________________________________________
    activation_14 (Activation)   (None, 13402)             0         
    =================================================================
    Total params: 17,431,130
    Trainable params: 17,431,130
    Non-trainable params: 0
    _________________________________________________________________
    '''
    
    
    
    
   
    batch_size=batch_size
    #print ('build model...')
    model=Sequential()
    model.add(LSTM(256,input_shape=(maxlen,len(char_indices)),recurrent_dropout=0.1,dropout=0.1))
    model.add(Dense(len(char_indices)))
    model.add(Activation('softmax'))
    
    optimizer=keras.optimizers.RMSprop(lr=0.01)
    model.compile(loss='categorical_crossentropy',optimizer=optimizer)
    
    #print (model.summary())
    
    return model


def train_model(X,y,batch_size=256,epochs=10):
    '''
    小批量训练fit_generator
    '''
    model.fit_generator(data_generator(X,y,batch_size),steps_per_epoch=X.shape[0]//batch_size,epochs=epochs)




def data_generator(X,y,batch_size):
    '''
    数据较小批量生成
    '''
    if batch_size<1:
        batch_size=256
    number_of_batchs=X.shape[0]/batch_size
    counter=0
    shuffle_index=np.arange(np.shape(y)[0])
    np.random.shuffle(shuffle_index)
    
    #reset generator
    while 1:
        index_batch=shuffle_index[batch_size*counter:batch_size*(counter+1)]
        X_batch=(X[index_batch,:,:]).astype('float32')
        y_batch=(y[index_batch,:]).astype('float32')
        counter+=1
        yield(np.array(X_batch),y_batch)
        
        if (counter<number_of_batchs):
            np.random.shuffle(shuffle_index)
            counter=0
            
            
def gen_matrix_for_sentence():
    #对构造句子 对其矩阵化
    print ('vectorization...')
    X=np.zeros((len(sentences),maxlen,len(char_indices)),dtype=np.bool)
    y=np.zeros((len(sentences),len(char_indices)),dtype=np.bool)
    
    for i,sentence in enumerate(sentences[:]):
        if (i%30000==0):
            print (i)
            for t in range(maxlen):
                char_index=sentence[t]
                X[i,t,char_index]=1
    
        y[i,next_chars[i]]=1
    
    return X,y





        
        
if __name__=='__main__':
    
    #1、读文章数据
    #############alltext只要是文本即可 也可以留下文本标点###################
#    f=open('./fxhh_article.csv','r',encoding='utf-8')
#    line=f.readline()
#   alltext=''
#    i=0
#    while True:
#        line=f.readline()
#        if not line:break
#        i+=1
        
        #剔除标点符号
#        alltext+=line.split('\x01')[2]+" "+" ".join([x.strip() for x in re.findall('(.*?)[|,|;|?|.|!|:|，|。|、|~|；|！|？|"|“|”|】|【|《|》|<|>|&|%|$|(|)|（|）|^|#|@]', line.split('\x01')[3] + ",")])
        
        #防止内存溢出 在gen_matrix_for_sentence
#        if i>1000:break
    
    f=open('./7fresh_train_data_fruit.txt','r',encoding='utf-8')   
    alltext=''
    i=0
    while True:
        line=f.readline()
        if not line:break
        i+=1
        

        alltext+=f.readline().replace(" ","").strip()+" "       
        #防止内存溢出 在gen_matrix_for_sentence
        if i>15000:break
    ###############################################################
    
    #2、生成词与索引，索引与词的映射字典
    data_helper.preprocess_and_save_data(alltext, data_helper.create_lookup_tables) #生成并保存
    int_text, char_indices, indices_char = data_helper.load_preprocess() #加载 int_text即是包含所有词语索引号的数组
    
    #3、构造句子序列 int_text, vocab_to_int, int_to_vocab
    maxlen=4 #每个句子包含词语的最大个数
    step=2 #跳跃生成
    sentences=[]
    next_chars=[]
    for i in range(0,len(int_text)-maxlen,step):
        sentences.append(int_text[i:i+maxlen])
        next_chars.append(int_text[i+maxlen])
    
    print ('nb sequences:',len(sentences))

    #4、生成句子转矩阵  
    X,y=gen_matrix_for_sentence()
     
    #5、bulid模型   默认batch_size=256 小批量处理的句子树
    model=build_lstm_model(batch_size=256)
    
    #6、训练模型 epochs为训练轮次
    train_model(X,y,batch_size=256,epochs=20)
    
    
    #7、保存模型
    model.save('./lstm_model_gen_word_epoch_20.h5')
    
    

    




nb sequences: 44981
vectorization...
0
30000
Epoch 1/20
175/175 [==============================] - 238s - loss: 8.3286   
Epoch 2/20
175/175 [==============================] - 231s - loss: 7.0119   
Epoch 3/20
175/175 [==============================] - 231s - loss: 7.0095   
Epoch 4/20
175/175 [==============================] - 231s - loss: 6.9613   
Epoch 5/20
175/175 [==============================] - 231s - loss: 6.9561   
Epoch 6/20
175/175 [==============================] - 231s - loss: 6.9236   
Epoch 7/20
175/175 [==============================] - 231s - loss: 6.9447   
Epoch 8/20
175/175 [==============================] - 231s - loss: 6.9940   
Epoch 9/20
175/175 [==============================] - 231s - loss: 6.9796   
Epoch 10/20
175/175 [==============================] - 231s - loss: 6.9919   
Epoch 11/20
175/175 [==============================] - 231s - loss: 7.0645   
Epoch 12/20
175/175 [==============================] - 231s - loss: 7.1718   
Epoch 13/20
175/175 [=======

In [5]:
f=open('./7fresh_train_data_fruit.txt','r',encoding='utf-8')

In [16]:
alltext[:200]

'据说是汉朝张骞出使西域时由中亚经丝绸之路带入我国的 葡萄的含糖量达8%～10% 维生素及多种具有生理功能的物质 葡萄中的糖易被人体吸收 能很快被人体吸收 葡萄能比阿斯匹林更好地阻止血栓形成 降低血小板的凝聚力 葡萄中含的类黄酮是一种强力抗氧化剂 并可清除体内自由基 有帮助消化的作用 对人体裨益甚大 葡萄中含有一种有益的抗癌物质 并能防止癌细胞扩散 促进早日康复 葡萄性平味甘 生津液 通利小便的作用'

In [22]:
f=open('./7fresh_train_data_fruit.txt','r',encoding='utf-8')   
alltext=''
i=0
while True:
    line=f.readline()
    if not line:break
    i+=1


    alltext+=f.readline().replace(" ","").strip()+" "       


#2、生成词与索引，索引与词的映射字典
data_helper.preprocess_and_save_data(alltext, data_helper.create_lookup_tables) #生成并保存
int_text, char_indices, indices_char = data_helper.load_preprocess() #加载 int_te


wf=open("sentence_index.csv","w")

maxlen=10 #每个句子包含词语的最大个数
step=3 #跳跃生成
sentences=[]
next_chars=[]
for i in range(0,len(int_text)-maxlen,step):
    wf.write(str(int_text[i:i+maxlen]).strip("[]")+"\t"+str(int_text[i+maxlen])+"\n")

wf.close()


In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation,LSTM
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import keras
import numpy as np
#from keras.datasets import imdb
from keras import backend as K
import re
import jieba
import pickle
import data_helper

int_text, char_indices, indices_char = data_helper.load_preprocess() #加载 int_te
def process_line(line):  
    tmp_x = [int(val) for val in line.strip().split('\t')[0].split(",")]
    tmp_y=int(line.strip().split('\t')[1])
    
    x=np.zeros((maxlen,len(char_indices)),dtype=float)
    y=np.zeros(len(char_indices),dtype=float)
   
    for t in range(maxlen):
        char_index=tmp_x[t]
        x[t,char_index]=1
    
    y[tmp_y]=1


    return x,y  

def generate_arrays_from_file(path,batch_size):  
    while 1:  
        f = open(path)  
        cnt = 0  
        X =[]  
        Y =[]  
        for line in f:  
            # create Numpy arrays of input data  
            # and labels, from each line in the file  
            x, y = process_line(line)  
            X.append(x)  
            Y.append(y)
        
            cnt += 1  
            if cnt==batch_size:  
                cnt = 0  
                yield (np.array(X), np.array(Y)) 

                
                X = []  
                Y = []

    f.close()  

Using TensorFlow backend.


In [2]:
maxlen=10
def build_lstm_model(batch_size=256):

    batch_size=batch_size
    #print ('build model...')
    model=Sequential()
    model.add(LSTM(256,input_shape=(maxlen,len(char_indices)),recurrent_dropout=0.1,dropout=0.1))
    model.add(Dense(len(char_indices)))
    model.add(Activation('softmax'))
    
    optimizer=keras.optimizers.RMSprop(lr=0.01)
    model.compile(loss='categorical_crossentropy',optimizer=optimizer)
    
    #print (model.summary())
    
    return model
model=build_lstm_model(batch_size=100)
print (model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256)               44402688  
_________________________________________________________________
dense_1 (Dense)              (None, 43105)             11077985  
_________________________________________________________________
activation_1 (Activation)    (None, 43105)             0         
Total params: 55,480,673
Trainable params: 55,480,673
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
model.fit_generator(generate_arrays_from_file("./sentence_index.csv",50),steps_per_epoch=1000,epochs=20,verbose=1,nb_worker=1)

/home/jianxiaorong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """Entry point for launching an IPython kernel.
/home/jianxiaorong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=1000, epochs=20, verbose=1, workers=1)`
  """Entry point for launching an IPython kernel.


Epoch 1/20
   5/1000 [..............................] - ETA: 22335s - loss: 6.4309

/home/jianxiaorong/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (2.703873). Check your callbacks.
  % delta_t_median)


1000/1000 [==============================] - 2495s - loss: 6.3062  
Epoch 2/20
1000/1000 [==============================] - 2392s - loss: 8.3037  
Epoch 3/20
1000/1000 [==============================] - 2394s - loss: 8.6284  
Epoch 4/20
1000/1000 [==============================] - 2395s - loss: 8.9761  
Epoch 5/20
1000/1000 [==============================] - 2395s - loss: 8.2446  
Epoch 6/20
1000/1000 [==============================] - 2395s - loss: 8.5986  
Epoch 7/20
1000/1000 [==============================] - 2395s - loss: 8.4997  
Epoch 8/20
1000/1000 [==============================] - 2394s - loss: 8.6378  
Epoch 9/20
1000/1000 [==============================] - 2394s - loss: 7.9685  
Epoch 10/20
1000/1000 [==============================] - 2395s - loss: 8.2732  
Epoch 11/20
1000/1000 [==============================] - 2396s - loss: 8.1703  
Epoch 12/20
1000/1000 [==============================] - 2396s - loss: 8.1942  
Epoch 13/20
1000/1000 [==============================] - 239

In [6]:
model.save('./lstm_model_gen_word_epoch_20.h5')

In [9]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.